# 🐍 Setup Focoos

In [ ]:
%uv pip install -e "..[cpu]" # you can choose to install only the inference execution providers you need

# 🤖 Playground with Focoos pre-trained models


### Supported Tasks: Object Detection, Instance Segmentation, Semantic Segmentation

In [ ]:
import os
from pprint import pprint

from focoos import Focoos
from focoos.ports import DEV_API_URL

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

pprint(focoos.list_focoos_models())

## Remote Inference
This section demonstrates how to perform remote inference using a model from the Focoos platform.
We will load a remote model (can be a pre-trained model or a custom user model), and then run inference on a sample image with focoos API.


In [ ]:
import os

from supervision import plot_image

from focoos import Focoos

model_ref = "fai-rtdetr-m-obj365"
image_path = "./assets/ade_val_034.jpg"

# focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"))

model = focoos.get_remote_model(model_ref)

output, preview = model.infer(image_path, threshold=0.6, annotate=True)
plot_image(preview)

## Local Inference

This section demonstrates how to perform local inference using a model from the Focoos platform. 
We will load a model, deploy it locally, and then run inference on a sample image.


### 🔧 Available Runtimes and Execution Providers

#### To install the extras modules, use the command: `uv pip install .[{{extra-name}}]`.


- **TORCHSCRIPT** *`[torch]`*
    - **CUDA**
        NVIDIA GPU acceleration with FP32 precision, excellent balance between speed and accuracy, suggested for segmentation tasks
- **ONNXRUNTIME** providers:
    - **CPU** and **CoreML (Apple)**  *`base sdk without any extra`*
        Standard CPU execution, compatible with all systems but with limited performance
    - **CUDA** *`[cuda]`*

        NVIDIA GPU acceleration with FP32 precision, excellent balance between speed and accuracy
    - **TensorRT** *`[tensorrt]`*

        Maximum optimization for NVIDIA GPUs with reduced FP16 precision, exceptional performance, but slow model warmup due to tensorrt engine compilation




In [ ]:
from focoos.ports import RuntimeTypes

for runtime_type in RuntimeTypes:
    print(runtime_type)

### 🖥️  System info



In [ ]:
from focoos.utils.system import get_system_info

system_info = get_system_info()
system_info.pretty_print()

### Inference with TorchscriptRuntime (CUDA32)

In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import Focoos, RuntimeTypes

# focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"))
image_path = "./assets/ade_val_034.jpg"
model_ref = "fai-m2f-s-ade"


model = focoos.get_local_model(model_ref, runtime_type=RuntimeTypes.TORCHSCRIPT_32)

latency = model.benchmark(iterations=10, size=640)
pprint(latency)

output, preview = model.infer(image_path, threshold=0.6, annotate=True)
pprint(output.detections)
pprint(output.latency)

plot_image(preview)

### Inference with OnnxRuntime (CUDA32)

In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import DEV_API_URL, Focoos, RuntimeTypes

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)
image_path = "./assets/ade_val_034.jpg"
model_ref = "fai-rtdetr-m-obj365"


model = focoos.get_local_model(model_ref, runtime_type=RuntimeTypes.ONNX_CUDA32)

latency = model.benchmark(iterations=10, size=640)
pprint(latency)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.6, annotate=True)
pprint(output.detections)
pprint(output.latency)

plot_image(preview)

### Inference with OnnxRuntime (TensorRT) (FP16)

In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import Focoos
from focoos.ports import RuntimeTypes

focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
)
image_path = "./assets/ade_val_034.jpg"
model_ref = "fai-rtdetr-m-obj365"


model = focoos.get_local_model(model_ref, runtime_type=RuntimeTypes.ONNX_TRT16)

latency = model.benchmark(iterations=10, size=640)
pprint(latency)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.6, annotate=True)
pprint(output.detections)
pprint(output.latency)

plot_image(preview)

# User Models

### List User Models

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from focoos import DEV_API_URL, Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

models = focoos.list_models()
pprint(models)

## Create Model

In [ ]:
import os
from pprint import pprint

from dotenv import load_dotenv

from focoos import DEV_API_URL, Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.new_model(name="test-model", focoos_model="fai-rtdetr-m-obj365", description="Test model")
### Get Model Info

## Cloud Inference

In [ ]:
import os
from pprint import pprint

import supervision as sv
from dotenv import load_dotenv

from focoos import Focoos

load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.get_remote_model(os.getenv("FOCOOS_MODEL_REF"))
model_info = model.get_info()
image_path = "./assets/aquarium.jpg"

output, preview = model.infer(image_path, threshold=0.5, annotate=True)
sv.plot_image(preview)

## Local Inference




In [ ]:
import os
from pprint import pprint

from supervision import plot_image

from focoos import DEV_API_URL, Focoos

focoos = Focoos(
    api_key=os.getenv("FOCOOS_API_KEY"),
    host_url=DEV_API_URL,
)
image_path = "./assets/aquarium.jpg"
model_ref = "focoos_object365"

model_info = focoos.get_model_info(model_ref)

model = focoos.get_local_model(model_ref)

latency = model.benchmark(iterations=10, size=640)
# pprint(latency)
output, preview = model.infer(image_path, threshold=0.3, annotate=True)

plot_image(preview)

## Train a Model

### List Public Datasets


In [ ]:
import os
from pprint import pprint

from focoos import DEV_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

datasets = focoos.list_shared_datasets()
pprint(datasets)
dataset_ref = "f92ac3b33b284a80"

In [ ]:
import os

from focoos.ports import Hyperparameters, TrainInstance

model = focoos.new_model(name="test-model-5", focoos_model="fai-rtdetr-s-coco", description="Test model 4")
model_ref = model.model_ref
res = model.train(
    dataset_ref=dataset_ref,
    instance_type=TrainInstance.ML_G4DN_XLARGE,
    volume_size=50,
    max_runtime_in_seconds=36000,
    hyperparameters=Hyperparameters(
        learning_rate=0.0001,
        batch_size=8,
        max_iters=5000,
        eval_period=100,
        resolution=640,
    ),  # type: ignore
)

### Monitor Training

In [ ]:
model.notebook_monitor_train(interval=30, plot_metrics=True)

In [ ]:
logs = model.train_logs()
pprint(logs)

### Model Metrics




In [ ]:
import os
from pprint import pprint

from focoos import DEV_API_URL, Focoos

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=DEV_API_URL)

model = focoos.get_remote_model("fai-m2f-s-ade")
metrics = model.metrics()
pprint(metrics)

### Visualize Metrics


In [ ]:
from focoos.utils.metrics import MetricsVisualizer

metrics_visualizer = MetricsVisualizer(metrics)
metrics_visualizer.log_metrics()
metrics_visualizer.notebook_plot_training_metrics()